# AutoGen værktøjsbrugseksempel


## Importer de nødvendige pakker


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Oprettelse af klienten

I dette eksempel vil vi bruge [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for adgang til LLM.

`model` er defineret som `gpt-4o-mini`. Prøv at ændre modellen til en anden model, der er tilgængelig på GitHub Models-markedspladsen, for at se forskellige resultater.

Som en hurtig test vil vi blot køre en simpel prompt - `Hvad er hovedstaden i Frankrig`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definering af funktionerne

I dette eksempel vil vi give agenten adgang til et værktøj, der er en funktion med en liste over tilgængelige feriedestinationer og deres tilgængelighed.

Du kan forestille dig, at dette kunne være et scenarie, hvor en rejseagent har adgang til en rejse-database, for eksempel.

Mens du gennemgår dette eksempel, er du velkommen til at prøve at definere nye funktioner og værktøjer, som agenten kan kalde.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definere Funktionværktøjet 
For at få agenten til at bruge `vacation_destinations` som et `FunctionTool`, skal vi definere det som et.

Vi vil også give en beskrivelse af værktøjet, som hjælper agenten med at forstå, hvad værktøjet bruges til i forhold til den opgave, brugeren har anmodet om.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definere agenten

Nu kan vi oprette agenten i nedenstående kode. Vi definerer `system_message` for at give agenten instruktioner om, hvordan den kan hjælpe brugere med at finde feriedestinationer.

Vi sætter også parameteren `reflect_on_tool_use` til true. Dette gør det muligt for LLM at tage svaret fra værktøjskaldet og sende svaret ved hjælp af naturligt sprog.

Du kan sætte parameteren til false for at se forskellen.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Køre agenten

Nu kan vi køre agenten med den første besked fra brugeren, der spørger om at tage en tur til Tokyo.

Du kan ændre denne bydestination for at se, hvordan agenten reagerer på tilgængeligheden af byen.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på at opnå nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
